In [5]:
import numpy as np
import os
from typing import List
import nibabel as nib
import pandas as pd
import h5py

In [ ]:
def maybe_mkdir_p(directory: str) -> None:
    os.makedirs(directory, exist_ok=True)

base_dir = os.getcwd()
input_dir = '/mnt/NM/dataset/'

try:
    import nnunet
except:
    respository_dir = os.path.join(base_dir, 'models/module')
    os.chdir(respository_dir)
    ! pip install -e .
    os.chdir(base_dir)

# from scipy import special
# import copy

In [7]:
# Utils ..
def subdirs(folder: str, join: bool = True, prefix: str = None, suffix: str = None, sort: bool = True) -> List[str]:
    if join:
        l = os.path.join
    else:
        l = lambda x, y: y
    res = [l(folder, i) for i in os.listdir(folder) if os.path.isdir(os.path.join(folder, i))
           and (prefix is None or i.startswith(prefix))
           and (suffix is None or i.endswith(suffix))]
    if sort:
        res.sort()
    return res


def subfiles(folder: str, join: bool = True, prefix: str = None, suffix: str = None, sort: bool = True) -> List[str]:
    if join:
        l = os.path.join
    else:
        l = lambda x, y: y
    res = [l(folder, i) for i in os.listdir(folder) if os.path.isfile(os.path.join(folder, i))
           and (prefix is None or i.startswith(prefix))
           and (suffix is None or i.endswith(suffix))]
    if sort:
        res.sort()
    return res

In [ ]:
# Folder : 

! nnUNet_ensemble -f ./21-11-18_16:29:27-models/result/ ./21-11-18_17:01:56-models/result/ -o ./models/result/

In [4]:
size_dic = {}

def hdf2nifti_size(hdf_folder: str):
    # hdf_folder : [train_dir, test_dir] hdf5 file path
    # save_folder : [imagesTr, imagesTs] Save Folder path

    hdf5_files = os.listdir(hdf_folder)
    hdf5_files.sort()


    for hdf5_file in hdf5_files:

        hdf5_path = os.path.join(hdf_folder, hdf5_file)

        # image
        f_i = h5py.File(hdf5_path, 'r')
        ctarr = np.asarray(f_i['CT'])
        petarr = np.asarray(f_i['PET'])
        sizearr = np.asarray(f_i['Size'])
        f_i.close()
        
        size_dic[hdf5_file[:17]] = sizearr # new

        print('{} = sizearr : {}, ctarr : {}, petarr : {}'.format(hdf5_file, sizearr, ctarr.shape, petarr.shape))


    print('Image Patient : {}'.format(len(os.listdir(hdf_folder))))


def get_suv_params(ptarr, roi):
    roi = np.asarray(roi>0, dtype=np.float)
    suvmax = np.max(ptarr*roi)
    suvmean = np.sum(ptarr*roi)/np.sum(roi)
    return suvmax, suvmean

def get_vol_params(ptzoom, roi):
    roi = np.asarray(roi>0, dtype=np.float)
    return np.prod(ptzoom) * np.sum(roi)

hdf2nifti_size(input_dir)

In [ ]:
PN_list = subfiles('./models/result/', join=False, suffix='.nii.gz')

for pn_l in PN_list:
    _, ext = os.path.splitext(pn_l)
    if ext == '.gz':
        
        pn = pn_l[:-7]

        ptarr = np.array(nib.load('./21-11-16_14:39:41-models/temp/convert_data/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs/{}_0001.nii.gz'.format(pn)).dataobj)
        ctarr = np.array(nib.load('./21-11-16_14:39:41-models/temp/convert_data/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs/{}_0000.nii.gz'.format(pn)).dataobj)
        pred_arr = np.array(nib.load('./models/result/{}.nii.gz'.format(pn)).dataobj)
        
        
        size = size_dic[pn]
        
        #Calculate Mean SUV and Max SUV

        suvmax, suvmean = get_suv_params(ptarr, pred_arr)

        #Calculate Volume

        aorvol = get_vol_params(size, pred_arr)
        
        data = {'case' : [pn], 'PD_Aorta_volume' : [aorvol], 'PD_SUVmean' : [suvmean]}
        df = pd.DataFrame(data)

        # .to_csv 
        if not os.path.exists('submission.csv'):
            df.to_csv('submission.csv', index=False, mode='w')
        else:
            df.to_csv('submission.csv', index=False, mode='a', header=False)
